In [8]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob

In [9]:
IMAGE_SIZE = [224, 224]
train_path = './drive/MyDrive/Facial Recognition/Datasets/train'
valid_path = './drive/MyDrive/Facial Recognition/Datasets/test'

In [10]:
# adding preprocessing layer to the front of VGG
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [11]:
# skipping the existing weights to train
for layer in vgg.layers:
  layer.trainable = False

In [12]:
# getting the total number of classes using glob.
folders = glob('./drive/MyDrive/Facial Recognition/Datasets/train/*')

In [13]:
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [14]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [15]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [16]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('./drive/MyDrive/Facial Recognition/Datasets/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('./drive/MyDrive/Facial Recognition/Datasets/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 618 images belonging to 4 classes.
Found 120 images belonging to 4 classes.


In [17]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
20/20 [==============================] - 512s 25s/step - loss: 2.1234 - accuracy: 0.4086 - val_loss: 0.7161 - val_accuracy: 0.7833
Epoch 2/5
20/20 [==============================] - 467s 23s/step - loss: 0.2820 - accuracy: 0.9062 - val_loss: 0.2503 - val_accuracy: 0.9000
Epoch 3/5
20/20 [==============================] - 466s 23s/step - loss: 0.0603 - accuracy: 0.9791 - val_loss: 0.4017 - val_accuracy: 0.8417
Epoch 4/5
20/20 [==============================] - 469s 24s/step - loss: 0.0354 - accuracy: 0.9973 - val_loss: 0.1921 - val_accuracy: 0.9250
Epoch 5/5
20/20 [==============================] - 469s 24s/step - loss: 0.0235 - accuracy: 0.9946 - val_loss: 0.1758 - val_accuracy: 0.9250


In [18]:
model.save('./drive/MyDrive/Facial Recognition/VGG19_model.h5')